### **Creating Workflow Templates**

This notebook will show how to create workflow templates and how to use them in a workflow.

**First some imports and global settings**

In [ ]:
from hera.workflows import Task, script, Parameter, DAG, Workflow, WorkflowTemplate
from hera.workflows.models import TemplateRef
from hera.shared import global_config

global_config.host = "https://services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"
global_config.image = "ghcr.io/eodcgmbh/cluster_image:2025.2.0"

**Writing scripts**

In [ ]:
@script()
def write_message(message):
    print(message)

In [ ]:
@script()
def echo_message(message):
    print(f"echo of {message}")

**Creating a Workflow Template**

Equivalent to creating a workflow, a workflow template can be created. As we want it to be possible to give parameter values to the function when using the template, we have to define `inputs` in the DAG, here we will use Parameters called *args* which we call when using the corresponding function. We can also pass multiple inputs if needed.

In [ ]:
with WorkflowTemplate(
    name="write-message-template",
    entrypoint="workflow"
) as wt:
    with DAG(name="workflow", inputs=Parameter(name="args")):
        step1 = write_message(arguments={"message": "{{inputs.parameters.args}}"})
        step2 = echo_message(arguments={"message": step1.result})

        step1 >> step2

**Submitting the Workflow Template**

In [ ]:
wt.create()

**Creating a Workflow using the Template**

To use the template we need to create a `Task` calling the relevant template with `template_ref` and passing the arguments to be used in the template with the correct name as defined in the template.

In [ ]:
with Workflow(
    generate_name="use-template-",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        step1 = write_message(arguments={"message": "We will use a template in the next step!"})
        step2 = Task(name="using-template", template_ref=TemplateRef(name="write-message-template", template="workflow"), arguments={"args": "Hello hera!"})

        step1 >> step2

**Submitting the Workflow**

In [ ]:
w.create()